<a href="https://colab.research.google.com/github/maetthil/NASASpaceApps/blob/main/Read_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# %%capture
!pip install cdflib
!pip install pyspedas
!pip install matplotlib==3.1.2
!pip install pyarrow

IOStream.flush timed out
IOStream.flush timed out
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 2.7 MB/s eta 0:00:0000:0100:01


In [ ]:
import calendar
import datetime
from typing import List, Tuple, NoReturn
import glob
import os
import re
import sys

import matplotlib.pyplot as plt
import numpy as np
import requests
import pandas as pd
import pytz
import pyspedas
import pytplot
from pytplot import tplot
from tqdm import tqdm

: 


## Magnetic Field

In [29]:
def getMagneticField(
    satellite: str = "wind", 
    trange: List[str] = ['2018-01-01',
                         '2018-01-10']) -> Tuple[pd.DataFrame, str]:
    """Realiza o download do conjunto de dados de campo magnético.
    
    Args:
        satellite: Satélite que foi usado para capturar os resultados.
        trange: Uma lista de strings representando o intervalo fechado 
          usado para filtrar os dados.
    Return:
        Uma tupla representando uma instância de `pd.DataFrame` e uma 
          string com o caminho onde será armazenado.
    """
    pytplot.del_data()
    if satellite == "wind":
        vars = pyspedas.wind.mfi(trange=trange)
        quants_mag = pytplot.data_quants['BGSE']
        mag = quants_mag.values
    else:
        mag = pyspedas.dscovr.mag(trange=trange)
        quants_mag = pytplot.data_quants['dsc_h0_mag_B1GSE']
        mag = quants_mag.values
        mag[mag == -1e+31] = np.nan

    time_mag = quants_mag.coords['time'].values
    time_mag = [datetime.datetime.fromtimestamp(
        i, pytz.timezone("UTC")) for i in time_mag]

    df_mag = pd.DataFrame({
        "Bx_GSE": mag[:,0], 
        "By_GSE": mag[:,1],
        "Bz_GSE": mag[:,2]}, index=time_mag)
    filePath = "mag"
    return df_mag, filePath

## Ion Plasma

In [30]:
def getIonPlasma(
    satellite: str = "wind",
    trange: List[str] = ['2018-01-01', 
                         '2018-01-10']) -> Tuple[pd.DataFrame, str]:
    """Realiza o download do conjunto de dados de plasmas.
    
    Args:
        satellite: Satélite que foi usado para capturar os resultados.
        trange: Uma lista de strings representando o intervalo fechado 
          usado para filtrar os dados.
    Return:
        Uma tupla representando uma instância de `pd.DataFrame` e uma 
          string com o caminho onde será armazenado.
    """
    pytplot.del_data()
    if satellite == "wind":
        vars = pyspedas.wind.swe(trange=trange)
        quants_vGSE = pytplot.data_quants['U_eGSE']
        quants_therSp = pytplot.data_quants['T_elec']
        quants_Np = pytplot.data_quants['N_elec']

        vGSE = quants_vGSE.values
        therSp = quants_therSp.values
        Np = quants_Np.values
    else:
        vars = pyspedas.dscovr.fc(trange=trange)

        quants_vGSE = pytplot.data_quants['dsc_h1_fc_V_GSE']
        quants_therSp = pytplot.data_quants['dsc_h1_fc_THERMAL_SPD']
        quants_Np = pytplot.data_quants['dsc_h1_fc_Np']

        vGSE = quants_vGSE.values
        therSp = quants_therSp.values
        Np = quants_Np.values

        vGSE[vGSE == -1e+31] = np.nan
        therSp[therSp == -1e+31] = np.nan
        Np[Np == -1e+31] = np.nan

    time_fc = quants_vGSE.coords['time'].values
    time_fc = [datetime.datetime.fromtimestamp(
        i, pytz.timezone("UTC")) for i in time_fc]

    df_ions = pd.DataFrame({
        "Vx_GSE": vGSE[:,0], 
        "Vy_GSE": vGSE[:,1],
        "Vz_GSE": vGSE[:,2],
        "Therm_spd": therSp,
        "Np":Np}, index = time_fc)
    filePath = "ions"
    return df_ions, filePath

In [ ]:
ions = getIonPlasma(satellite="dscovr", trange=['2018-01-01', '2018-01-10'])
ions.head()

In [ ]:
mag = getMagneticField(satellite="dscovr", trange=['2018-01-01', '2018-01-10'])
mag.head()

In [35]:
def downloadSaveParquet(
    downloadFunc: callable,
    sat: str,
    yearInterval: List[int],
    monthInterval: List[int]) -> NoReturn:
    """Transforma um arquivo .CDF em um arquivo .parquet.
    
    Args:
        downloadFunc: Função de download de um conjunto de dados.
        sat: Satélite que foi usado para capturar os resultados.
        yearInterval: Uma intevalo fechado de anos para filtrar o 
          conjunto de dados.
        monthInterval: Uma intevalo fechado de meses para filtrar o 
          conjunto de dados.
    """
    for yr in tqdm(range(yearInterval[0], yearInterval[1])):
        for mth in range(monthInterval[0], monthInterval[1]):
            lastDay = calendar.monthrange(yr, mth)[1]
            try:
                range_list = [f'{yr:04d}-{mth:02d}-01', f'{yr:04d}-{mth:02d}-{lastDay:02d}']
                output, filepath = downloadFunc(satellite=sat, trange=range_list)
                export_path = f'./parquetData/{filepath}_{yr:04d}-{mth:02d}_{sat}.parquet'
                output.to_parquet(export_path, engine="pyarrow")
            except Exception as e:
                print("Variavel nao encontrada")



In [ ]:
# 18h por instrumento, de cada satélites (para 6 anos) 
downloadSaveParquet(getMagneticField, sat="dscvr", yearInterval=[2017, 2018], monthInterval=[1, 7])

In [ ]:
# dados de ions do plasma 18h por instrumento, para cada sat[elite]
downloadSaveParquet(getIonPlasma, sat="dscvr", yearInterval=[2017, 2018], monthInterval=[1, 7])

In [ ]:
downloadSaveParquet(getMagneticField, sat="wind", yearInterval=[2017, 2018], monthInterval=[1, 7])

In [ ]:
downloadSaveParquet(getIonPlasma, sat="wind", yearInterval=[2017, 2018], monthInterval=[1, 7])

## Série temporal 

In [ ]:
# A lot of ideas, but little data for processing.